In [31]:
import tensorflow as tf
import keras
#from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.models import Model
from keras.layers import *
import os
import sys
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
base_dir = "C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/score_pics/"
train_dir = base_dir + "train_pics/"
validation_dir = base_dir + "validation_pics/"
test_dir = base_dir + "test_pics/"

In [4]:
print("total training images:", len(os.listdir(train_dir)))
print("total validation images:", len(os.listdir(validation_dir)))
print("total test images:", len(os.listdir(test_dir)))



total training images: 37
total validation images: 14
total test images: 16


## Keras Sequential API

In [31]:
model = ks.models.Sequential()
model.add(layers.Conv2D(16, (5,5), activation = "relu", input_shape=(160, 50, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32, (5,5), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (5,5), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 156, 46, 16)       1216      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 78, 23, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 74, 19, 32)        12832     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 37, 9, 32)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 33, 5, 64)         51264     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 2, 64)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2048)              0         
Total para

In [ ]:
model.compile(loss="binary_crossentropy",
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])

In [35]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(160, 50),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(160, 75),
        batch_size=20,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(160, 75),
        batch_size=20,
        class_mode='binary')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


## Keras functional API

In [50]:
train_folder = base_dir + "train_pics"
valid_folder = base_dir + "validation_pics"
test_folder = base_dir + "test_pics"

In [51]:
train_names = [f for f in os.listdir(train_folder)]
valid_names = [f for f in os.listdir(valid_folder)]
test_names = [f for f in os.listdir(test_folder)]

In [52]:
print(str(len(train_names)) + " training files")
print(str(len(valid_names)) + " validation files")
print(str(len(test_names)) + " test files")

29 training files
14 validation files
16 test files


In [34]:
train_dataset = np.ndarray(shape=(len(train_names), 50, 160, 3), dtype = np.float32)

i = 0
for f in train_names:
    img = load_img(folder + "/" + f)
    x = img_to_array(img)
    #x = x.reshape(())
    x = (x - 128) / 128
    dataset[i] = x
    i += 1

In [55]:
def create_dataset(fold, names):
    dataset = np.ndarray(shape=(len(train_names), 50, 160, 3), dtype = np.float32)
    i = 0
    for f in names:
        img = load_img(base_dir + fold + "/" + f)
        x = img_to_array(img)
        #x = x.reshape(())
        x = (x - 128) / 128
        dataset[i] = x
        i += 1
    return dataset

In [57]:
train_dataset = create_dataset("train_pics", train_names)
validation_dataset = create_dataset("validation_pics", valid_names)
test_dataset = create_dataset("test_pics", test_names)

In [19]:
#Filters:
f1, f2, f3 = 16,32,64
patch_size = 5
p_size = (2,2)

#TODO correct shape
inp = Input(shape=(75, 250,1))


conv1 = Conv2D(filters = f1, kernel_size = (3,3), activation = "relu")(inp)
maxp1 = MaxPool2D(pool_size = p_size)(conv1)
#tbc